In [ ]:
import numpy as np
import rasterio as rio
from rasterio.crs import CRS
from affine import Affine
from matplotlib import pyplot
%matplotlib inline
import os

In [ ]:
# Create array of ones size of a global, 1 degree raster
dst_array = np.ones((360, 180), dtype=np.float32)
#dst_array = np.random.randn(360, 180).astype(np.float32)
#print(dst_array)

# Use rasterio to classify this as WGS84 EPSG:4326
world_array = '/Users/nathansuberi/Desktop/RW_Data/world_array.tif'

profile = {
    'driver': 'GTiff', 
    'dtype': np.float32, 
    'nodata': 0, 
    'width': 360, 
    'height': 180, 
    'count': 1, 
    'crs': CRS({'init': 'EPSG:4326'}), 
    'transform':Affine(1, 0, -180, 0, -1, 90),
    'blockxsize': 128, 
    'blockysize': 128, 
    'tiled': True, 
    'compress': 'lzw', 
    'interleave': 'band'
}
with rio.open(world_array, "w", **profile) as dst:
    dst.write(dst_array, indexes=1)

# Print, observe
with rio.open(world_array, "r") as src:
    data = src.read(indexes=1)
    print(src.profile)
    print(data)
pyplot.imshow(data)

In [ ]:
# Define alternate projection
alt_proj = "EPSG:54009"

In [ ]:
# Use gdal to re-project as Mollweide EPSG:54009
os.environ["Zsrc_file"] = world_array
world_array_edit = world_array[:-4] + "_edit.tif"
os.environ["Zdst_file"] = world_array_edit
os.environ["Zoptions"] = "-r near -s_srs EPSG:4326 -t_srs "+alt_proj+" -of GTiff -overwrite"
!gdalwarp $Zoptions $Zsrc_file $Zdst_file

# Print, observe
with rio.open(world_array_edit, "r") as src:
    data = src.read(indexes=1)
    print(src.profile)
    print(data)
    print(np.mean(data))
    
pyplot.imshow(data)
!gdalinfo $Zdst_file

In [ ]:
# Use gdal to re-project as WGS84 EPSG:4326
os.environ["Zsrc_file"] = world_array_edit
world_array2 = world_array[:-4] + "2.tif"
os.environ["Zdst_file"] = world_array2 

os.environ["Zoptions"] = "-s_srs "+alt_proj+" -t_srs EPSG:4326 -of GTiff -overwrite"
os.environ["Zoptions_with_tr"] = "-r near -s_srs "+alt_proj+" -t_srs EPSG:4326 -of GTiff -overwrite -tr 1 1 -te -180 -90 180 90 -wo SOURCE_EXTRA=1000"
#!gdalwarp $Zoptions $Zsrc_file $Zdst_file
!gdalwarp $Zoptions_with_tr $Zsrc_file $Zdst_file

# Print, observe
with rio.open(world_array2, "r") as src:
    data = src.read(indexes=1)
    print(src.profile)
    print(data)
    print(np.mean(data))
    
pyplot.imshow(data)
!gdalinfo $Zdst_file

In [ ]:
# Use gdal to re-project as WGS84 EPSG:4326
os.environ["Zsrc_file"] = world_array
world_array3 = world_array[:-4] + "3.tif"
os.environ["Zdst_file"] = world_array3
os.environ["Zoptions"] = "-s_srs EPSG:4326 -t_srs EPSG:4326 -of GTiff -overwrite"
!gdalwarp $Zoptions $Zsrc_file $Zdst_file

# Print, observe
with rio.open(world_array3, "r") as src:
    data = src.read(indexes=1)
    print(src.profile)
    print(data)
    print(np.mean(data))
    
pyplot.imshow(data)
!gdalinfo $Zdst_file

In [ ]:
### Create areal weighted rasters
# https://waterprogramming.wordpress.com/2015/06/09/using-arcpy-to-calculate-area-weighted-averages-of-gridded-spatial-data-over-political-units-part-2/
# http://pythonhosted.org/rasterstats/cli.html
# http://mathforum.org/library/drmath/view/63767.html

# https://stackoverflow.com/questions/41826750/calculating-the-area-of-gridded-data-equidistant-in-degrees
# http://unidata.github.io/netcdf4-python/#section1
from netCDF4 import Dataset
import numpy as np
import os
from pyproj import Proj
# Didn't end up using this:
# from shapely.geometry import shape

def calc_cell_area(lon, lon_res, num_cols):
    # TO DO: Bring in Thomas' formula
    R = 6371.

    lon = lon*(np.pi/180)
    lon_res = lon_res*(np.pi/180)

    theta1 = lon
    theta2 = lon-lon_res
    
    h1 = R*np.sin(theta1)
    h2 = R*np.sin(theta2)
    
    A = 2*np.pi*R*np.abs(h1-h2) / num_cols
    
    #print("lon:", lon, ", area:", A)
    return(A)

def create_areal_raster(file_name, start_lat, start_lon, end_lat, end_lon, resolution, col_vector=True):
    # Create netcdf from scratch
    Data = Dataset(file_name, mode="w")

    row_range = (start_lat - end_lat)
    col_range = (end_lon - start_lon)
    
    num_cols = int(col_range / resolution)
    num_rows = int(row_range / resolution)

    Data.createDimension("dim_data_col_vector", num_rows*num_cols)
    #Data.createDimension("dim_data_2d_array", (num_rows, num_cols))

    col_lats = [np.repeat(i, num_rows) for i in np.arange(-180, 180, resolution)]
    row_lons = [np.arange(90, -90, -resolution)]*num_cols
    
    if col_vector:
        lats = Data.createVariable('latitude', 'f4', 'dim_data_col_vector')
        lons = Data.createVariable('longitude', 'f4', 'dim_data_col_vector')
        lats[:] = np.reshape(col_lats, -1)
        lons[:] = np.reshape(row_lons, -1)
        
        # Create a vector of areas for longitude = -180, over whole range of latitudes
        # Broadcast this to be size of AREA_VAR
        AREA_VAR = np.empty(len(lats))
        #projection = Proj(init="EPSG:4326")
        
        areas = []
        
        for j in np.arange(start_lat,end_lat,-resolution):  #just northern hemisphere
            areas.append(calc_cell_area(j,resolution,num_cols))
            
        #print(areas)
        areas = np.repeat(areas, num_cols)
        AREA_VAR = np.reshape(areas, -1)
        #print(AREA_VAR)
        
        cell_area = Data.createVariable('cell_area', 'f4', 'dim_data_col_vector')            
        cell_area[:] = AREA_VAR
        
    else:
        lats = Data.createVariable('latitude', 'f4', 'dim_data_2d_array')
        lons = Data.createVariable('longitude', 'f4', 'dim_data_2d_array')
        # TO DO: Implement 2d array version

    Data.close()
    
    
os.chdir("/Users/nathansuberi/Desktop/RW_Data/Areal_Rasters/")

raster_set = {
    "./1_degree_areal_raster.nc":1,
    "./30_arc_minute_areal_raster.nc":1/2.,
    "./15_arc_minute_areal_raster.nc":1/4.,
    "./5_arc_minute_areal_raster.nc":1/12.,
    "./3_arc_minute_areal_raster.nc":1/20.,
    "./1_arc_minute_areal_raster.nc":1/60.,
    "./30_arc_second_areal_raster.nc":1/120.,
    "./15_arc_second_areal_raster.nc":1/240.,
    "./5_arc_second_areal_raster.nc":1/720.,
    "./3_arc_second_areal_raster.nc":1/1200.,
    "./1_arc_second_areal_raster.nc":1/3600.,   
}

for file_name, resolution in raster_set.items():
    areal_rasters_args = {
        "file_name":file_name, 
        "start_lat": 90, 
        "start_lon": -180, 
        "end_lat": -90, 
        "end_lon": 180, 
        "resolution":resolution, 
        "col_vector":True
        }
    
    try:
        create_areal_raster(**areal_rasters_args)
        print("Done")
    except:
        print("File already exists - permission denied.")

Done
Done
Done
Done
Done
Done


In [ ]:
Read_data = Dataset("./test_areal_raster24.nc", "r+")
Read_data.variables["latitude"]
areas = Read_data.variables["cell_area"][:]
areas[1000:2000]

In [ ]:
# Add more data, now have lat, lon, and area of each cell
# Merge WFP Price data w/ shapefile
# Rasterize the shapefile
# Logistic regression on whether an area experiences a price spike
# Use NDVI and SPI in an area... maybe trade data? macroeconomic indicators? crop calendars?

# Add a masked layer using Logistics Cluster Global Obstacles data
import os
from hdx.hdx_configuration import Configuration
from hdx.data.dataset import Dataset
Configuration.create(hdx_site="test", hdx_read_only=True)

In [ ]:
import pprint

datasets = Dataset.search_in_hdx('Food and Commodity WFP', rows=10)
#print(pprint.pprint(datasets, depth=2))
#print(len(datasets))
#print(datasets[1])
# don't forget the square brackets here
resources = Dataset.get_all_resources( [datasets[1]] )
# or use the more concise
resource = Dataset.get_resources(datasets[0])

# includes data about a hash, so you can verify you have the up-to-date non-tampered data
print(resource)

# This downloads the dataset 
url, path = resource[0].download()
print('Resource URL %s downloaded to %s' % (url, path))

In [ ]:
import pandas as pd
pd.options.display.max_columns = 1000
pd.options.display.max_rows = 100

wfp_food_price_data = pd.read_csv(path,encoding = "ISO-8859-1")
df = wfp_food_price_data

In [ ]:
mkt_names = df.loc[:,"mkt_name"].unique()
print(mkt_names)
commodities = {}
for mkt in mkt_names:
    commodities[mkt] = df.loc[df.loc[:,"mkt_name"]==mkt, "cm_name"].unique()

In [3]:
print("Started")

from hdx.hdx_configuration import Configuration
from hdx.data.dataset import Dataset
#Configuration.create(hdx_site="test", hdx_read_only=True)

datasets = Dataset.search_in_hdx('Food and Commodity WFP', rows=10)
resource = Dataset.get_resources(datasets[0])
url, path = resource[0].download()

print("Data path retrieved")

import pandas as pd
pd.options.display.max_columns = 1000
pd.options.display.max_rows = 100

wfp_food_price_data = pd.read_csv(path,encoding = "ISO-8859-1")
df = wfp_food_price_data

print("Data loaded")

mkt_names = df.loc[:,"mkt_name"].unique()
print(mkt_names)
commodities = {}
for mkt in mkt_names:
    commodities[mkt] = df.loc[df.loc[:,"mkt_name"]==mkt, "cm_name"].unique()

print("Markets and Commodities Set")
    
from datetime import date
from sklearn import linear_model
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np

# can be used for the toordinal function
def create_ordinal_date_column(date_tuple):
    return(date.toordinal(date(*date_tuple)))

def add_dummy_columns(df, src_col):
    """Create additional columns on df as dummy variables 
        for values in src_col (i.e. months) """
    
    dummy_data = df.loc[:,(src_col)]
    dummy_vals = dummy_data.unique()
    
    for val in dummy_vals:
        dummy = (dummy_data == val).astype(int)
        df.loc[:,("month_"+str(val))] = dummy
    
    return(df)

def sanity_check(df):
    # TO DO: check whether there are at least 3 records for each month
    months = df["mp_month"]
    #print(months)
    ct_months = np.zeros(12)
    for month in months:
        ct_months[month-1] +=1
        
    #print(ct_months)
    for ct in ct_months:
        if ct < 3:
            return(False)
    return(True)

def calc_alps(dev):
    if(dev < .25):
        return("white")
    elif(dev < 1):
        return("yellow")
    elif(dev < 2):
        return("orange")
    else:
        return("red")

for mkt in mkt_names:
    for cmdty in commodities[mkt]:
        print("Market:", mkt, ", Commodity:", cmdty)
        selection = (df["mkt_name"]==mkt) & (df["cm_name"]==cmdty)
        
        price_history = df.loc[(selection), :]
        
        # Sanity check - at least 3 years of data for each month?
        if(not sanity_check(price_history)):
            print("Not enough raw training data")
            continue
        
        # Create ordinal date column
        date_nums = list(zip(price_history["mp_year"], price_history["mp_month"], np.ones(price_history.shape[0]).astype(int)))
        ordinal_dates = list(map(create_ordinal_date_column, date_nums))
        price_history.loc[:,("ordinal_dates")] = ordinal_dates
        
        # Create dummy columns for each month
        price_history = add_dummy_columns(price_history, "mp_month")
        
        # Create training and label data
        training_cols = ["ordinal_dates", "month_1", "month_2", "month_3",
                        "month_4", "month_5", "month_6",
                        "month_7", "month_8", "month_9",
                        "month_10", "month_11", "month_12"]
        X = price_history.loc[:, training_cols]
        Y = price_history.loc[:,"mp_price"]
        
        lm = linear_model.LinearRegression()
        lm.fit(X, Y)

        #print(lm.coef_)
        #print(lm.intercept_)
        
        # Calculate model residuals
        Y_hat = lm.predict(X)
        residuals = Y - Y_hat
        #print(residuals)
        
        # Divide by standard deviation of residuals
        resid_std_dev = np.sqrt(mean_squared_error(Y, Y_hat))
        ## ^ how to find from lm object?
        std_devs = residuals / resid_std_dev
        
        # Retrain model without first pass outliers
        price_history_tame = price_history.loc[(std_devs > -1) & (std_devs < 1), :]
        
        if(not sanity_check(price_history_tame)):
            print("Not enough tame training data")
            continue
        
        X_tame = price_history.loc[:,training_cols]
        Y_tame = price_history.loc[:,("mp_price")]
                
        lm_tame = linear_model.LinearRegression()
        lm_tame.fit(X_tame, Y_tame)
        
        Y_hat_tame = lm_tame.predict(X)
        residuals_tame = Y - Y_hat_tame
        #print(residuals_tame)
        
        # Divide by standard error of estimatation
        resid_std_dev_tame = np.sqrt(mean_squared_error(Y, Y_hat_tame))
        ## ^ how to find from lm object?
        std_devs_tame = residuals_tame / resid_std_dev_tame
        
        # Calculate ALPS
        ALPS = list(map(calc_alps, std_devs_tame))
        print("setting ALPS on current selection")
        df.loc[selection, "ALPS"] = ALPS
        print(type(list(lm_tame.coef_)))
        df.loc[selection, "Fitted Model"] = lm
        df.loc[selection, "Fitted Model Tame"] = lm_tame
        df.loc[selection, "Model Residual"] = std_devs_tame
        df.loc[selection, "Model Std Error of Residuals"] = resid_std_dev_tame

Started
Data path retrieved
Data loaded
['Fayzabad' 'Mazar' 'Bamyan' ..., 'Rumbek' 'Jau' 'Torit']
Markets and Commodities Set
Market: Fayzabad , Commodity: Bread


/Users/nathansuberi/.pyenv/versions/3.6.0/lib/python3.6/site-packages/pandas/core/indexing.py:337: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
/Users/nathansuberi/.pyenv/versions/3.6.0/lib/python3.6/site-packages/pandas/core/indexing.py:517: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s
/Users/nathansuberi/.pyenv/versions/3.6.0/lib/python3.6/site-packages/scipy/linalg/basic.py:1018: RuntimeWarning: internal gelsd driver lwork query error, required iwork dimension not returned. Th

Not enough tame training data
Market: Fayzabad , Commodity: Wheat
setting ALPS on current selection
<class 'list'>
Market: Fayzabad , Commodity: Rice (low quality)
setting ALPS on current selection
<class 'list'>
Market: Fayzabad , Commodity: Wage (qualified labour)
Not enough tame training data
Market: Fayzabad , Commodity: Livestock (sheep, one-year-old alive female)
Not enough raw training data
Market: Fayzabad , Commodity: Fuel (diesel)
setting ALPS on current selection
<class 'list'>
Market: Fayzabad , Commodity: Exchange rate
Not enough tame training data
Market: Fayzabad , Commodity: Wage (non-qualified labour, non-agricultural)
setting ALPS on current selection
<class 'list'>
Market: Mazar , Commodity: Bread
Not enough tame training data
Market: Mazar , Commodity: Wheat
setting ALPS on current selection
<class 'list'>
Market: Mazar , Commodity: Rice (low quality)
setting ALPS on current selection
<class 'list'>
Market: Mazar , Commodity: Wage (qualified labour)
Not enough tame 

In [24]:
wfp_food_price_data.head()

,adm0_id,adm0_name,adm1_id,adm1_name,mkt_id,mkt_name,cm_id,cm_name,cur_id,cur_name,pt_id,pt_name,um_id,um_name,mp_month,mp_year,mp_price,mp_commoditysource,ALPS,Fitted Model,Fitted Model Tame,Model Residual,Model Std Error of Residuals
0,1,Afghanistan,272,Badakhshan,266,Fayzabad,55,Bread,87,AFN,15,Retail,5,KG,1,2014,50.0,WFP,NaN,NaN,NaN,NaN,NaN
1,1,Afghanistan,272,Badakhshan,266,Fayzabad,55,Bread,87,AFN,15,Retail,5,KG,2,2014,50.0,WFP,NaN,NaN,NaN,NaN,NaN
2,1,Afghanistan,272,Badakhshan,266,Fayzabad,55,Bread,87,AFN,15,Retail,5,KG,3,2014,50.0,WFP,NaN,NaN,NaN,NaN,NaN
3,1,Afghanistan,272,Badakhshan,266,Fayzabad,55,Bread,87,AFN,15,Retail,5,KG,4,2014,50.0,WFP,NaN,NaN,NaN,NaN,NaN
4,1,Afghanistan,272,Badakhshan,266,Fayzabad,55,Bread,87,AFN,15,Retail,5,KG,5,2014,50.0,WFP,NaN,NaN,NaN,NaN,NaN


In [14]:
import pickle
df.to_pickle("/Users/nathansuberi/Desktop/RW_Data/wfp_alps.pkl")

In [18]:
df2 = pd.read_pickle("/Users/nathansuberi/Desktop/RW_Data/wfp_alps.pkl")

In [23]:
pd.options.display.max_rows = 1000
df2.head(1000)

,adm0_id,adm0_name,adm1_id,adm1_name,mkt_id,mkt_name,cm_id,cm_name,cur_id,cur_name,pt_id,pt_name,um_id,um_name,mp_month,mp_year,mp_price,mp_commoditysource,ALPS,Fitted Model,Fitted Model Tame,Model Residual,Model Std Error of Residuals
0,1,Afghanistan,272,Badakhshan,266,Fayzabad,55,Bread,87,AFN,15,Retail,5,KG,1,2014,5.000000e+01,WFP,NaN,NaN,NaN,NaN,NaN
1,1,Afghanistan,272,Badakhshan,266,Fayzabad,55,Bread,87,AFN,15,Retail,5,KG,2,2014,5.000000e+01,WFP,NaN,NaN,NaN,NaN,NaN
2,1,Afghanistan,272,Badakhshan,266,Fayzabad,55,Bread,87,AFN,15,Retail,5,KG,3,2014,5.000000e+01,WFP,NaN,NaN,NaN,NaN,NaN
3,1,Afghanistan,272,Badakhshan,266,Fayzabad,55,Bread,87,AFN,15,Retail,5,KG,4,2014,5.000000e+01,WFP,NaN,NaN,NaN,NaN,NaN
4,1,Afghanistan,272,Badakhshan,266,Fayzabad,55,Bread,87,AFN,15,Retail,5,KG,5,2014,5.000000e+01,WFP,NaN,NaN,NaN,NaN,NaN
5,1,Afghanistan,272,Badakhshan,266,Fayzabad,55,Bread,87,AFN,15,Retail,5,KG,6,2014,5.000000e+01,WFP,NaN,NaN,NaN,NaN,NaN
6,1,Afghanistan,272,Badakhshan,266,Fayzabad,55,Bread,87,AFN,15,Retail,5,KG,7,2014,5.000000e+01,WFP,NaN,NaN,NaN,NaN,NaN
7,1,Afghanistan,272,Badakhshan,266,Fayzabad,55,Bread,87,AFN,15,Retail,5,KG,8,2014,5.000000e+01,WFP,NaN,NaN,NaN,NaN,NaN
8,1,Afghanistan,272,Badakhshan,266,Fayzabad,55,Bread,87,AFN,15,Retail,5,KG,9,2014,5.000000e+01,WFP,NaN,NaN,NaN,NaN,NaN
9,1,Afghanistan,272,Badakhshan,266,Fayzabad,55,Bread,87,AFN,15,Retail,5,KG,10,2014,5.000000e+01,WFP,NaN,NaN,NaN,NaN,NaN


In [ ]:
df2.loc[df2["Model Std Error of Residuals"] > 10000,[""]]

In [ ]:
sq_res = np.power(residuals,2)
sum_sq_res = np.mean(sq_res)
print(mean_sq_error)
print(mean_squared_error(Y, Y_hat))

In [ ]:
# Calculate ALPS, reduce down to:
# 1: there is a price spike in this area, -or-
# 0: there is no price spike in this area, in this month.



commodity = "Bread"
fayz_data = wfp_food_price_data[(wfp_food_price_data["mkt_name"]==mkt_name) &
                               (wfp_food_price_data["cm_name"]==commodity)]
fayz_data




fayz_data = create_dummy_columns(fayz_data, "mp_month")

print(fayz_data.head())


lm = linear_model.LinearRegression()
X = fayz_data[["mp_month", "mp_year"]]
Y = fayz_data["mp_price"]
lm.fit(X,Y)

print(lm.predict(X)[0:5])
print(Y[0:5])
# Need to only train with data around these markets - don't train with
# areas that don't have the possibility of being recorded in the WFP price spike data

In [ ]:
int(False)

In [ ]:
a = np.array([[1,2,3], [3,4,5]])
print(a[1, np.newaxis])
print(a[1, None])
print(a[1,])

In [ ]:
a = np.array([[1,2,3,4,5,6,7,8,9,10]])
print(a[...,::-1])
print(a[:,...,np.newaxis])
print(a[(0,2)])

In [ ]:
import fiona
import geopandas as gpd

# Use geopandas or fiona to merge ALPS data with a shapefile - try GADM


In [ ]:
import rasterio as rio

# Use rasterio to rasterize the vector, print to a geotiff

In [ ]:
# Extract geotiff data, put into a NETCDF4 band


In [ ]:
# Create a machine learning architecture which uses this ALPS binary layer
# As labels for training data
# Training data:
## SPI
## NDVI
## Crop Calendars
## Irrigation Map
## MapSPAM

In [ ]:
# How to configure transforms for rasters??

def create_transform(row_width,row_rotation, upper_right_x,
                    column_rotation, column_height, upper_right_y):
    
    return(Affine(row_width,row_rotation,upper_right_x,
                  column_rotation, column_height, upper_right_y))

row_width=0
column_height=0
row_rotation=0
column_rotation=0
upper_right_x=-180
upper_right_y=90

degree1 = create_transform(1,0,-180,0,-1,90)
arcminutes30 = create_transform(.5,0,-180,0,-.5,90)
arcminutes15 = create_transform(.25,0,-180,0,-.25,90)
arcminutes5 = create_transform(0.0833333,0,-180,0,-0.0833333,90)
arcminutes3 = create_transform(row_width,0,-180,0,column_height,90)
arcminutes1 = create_transform(0.0166667,0,-180,0,-0.0166667,90)
arcseconds30 = create_transform(0.00833333,0,-180,-0,0.00833333,90)
arcseconds15 = create_transform(0.00416667,0,-180,-0,0.00416667,90)
arcseconds5 = create_transform(0.00138889,0,-180,0,-0.00138889,90)
arcseconds3 = create_transform(row_width,0,-180,0,column_height,90)
arcseconds1 = create_transform(0.0166667,0,-180,0,-0.0166667,90)

In [ ]:
### Use WFP Logistics Cluster data - create a raster
# 1 = WFP deployment to an area
# 0 = no WFP deployment to an area
# Divide up by month into many rasters, can use as training data labels
# Add in geocoded "Who does what where" from reliefweb
# Try to use GDELT as training data inputs

# Can 1Concern help here?

In [ ]:
# Fitting a gamma distribution to calculation SPI
# Other weighted moving average calculations over raster stacks